# Rolling Method 
Haoyang Han

get 35000 effective testing dataset.


Total method used here should be：
1. Standardization
2. nan value transfermation
3. feature selection via sharpe value
4. rolling method

#### 1. Import dataset

In [79]:
# 1. Import Dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()


import time
import os
import gc
import bottleneck as bn

# Define Hyperparameters
T = 30

import keras

In [78]:
# detailed keras function import

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

/Users/kevin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
address = os.listdir(r'../factors_pickingtime')
for path in address:
    print(type(path))
    print(path)
add = ['TSX[100001_2]', 'TSX[100035_1]', 'TSX[100041_4]', 'TSX[100041_1]','TSX[100041_2]','TSX[100041_3]']

<class 'str'>
TSX[100042_7]
<class 'str'>
TSX[100038_3]
<class 'str'>
TSX[100042_4]
<class 'str'>
TSX[100010_2]
<class 'str'>
TSX[100038_4]
<class 'str'>
TSX[100042_3]
<class 'str'>
TSX[100010_1]
<class 'str'>
TSX[100042_2]
<class 'str'>
TSX[100042_1]
<class 'str'>
TSX[100002_1]
<class 'str'>
TSX[100036_1]
<class 'str'>
TSX[100012_1]
<class 'str'>
TSX[100029_1]
<class 'str'>
TSX[100009_1]
<class 'str'>
TSX[100029_2]
<class 'str'>
TSX[100038_2]
<class 'str'>
TSX[100042_5]
<class 'str'>
TSX[100018_1]
<class 'str'>
TSX[100038_1]
<class 'str'>
TSX[100042_6]


6_variable Analysis

In [41]:
def import_dataset():
    '''
    read all 2-byte files and then concentrating them together. Follow this link:
    https://blog.csdn.net/brucewong0516/article/details/79062340
    
    instead of using alpha as y,
    we should use return rate as y.
    '''
    x = []
    y = []
    # import x
    address = os.listdir(r'../factors_pickingtime')
    u = 1
    for path in address:
        # if path in add:
        parent = r'../factors_pickingtime'
        abs_path = os.path.join(parent, path)
        x_i = np.memmap(abs_path,dtype = np.float32, shape = (40000,1829))# ,shape = (204800, 1749))
        x_i = np.asarray(x_i)
        x_ = x_i[:,0:1082]
        # x_ = x_i.reshape(-1,1)
        print(np.shape(x_))
        print("%d th factor have %d number of nan factors "%(u, np.isnan(x_).sum()))
        x.append(x_)
        # print(abs_path)
        u += 1
    # import y
    price = np.memmap('../quotes/stk_clsadj',dtype = np.float32, shape = (40000,1829))# ,shape = (204800, 1749))
    
    y = np.zeros((price.shape[0]-T,price.shape[1]))
    '''
    This is an iteration to create target y function;
    Here we use Return rate minus Average Return Rate.

    First we calculate the mean return rate for every single timeslot;
    Then we calculate the return rate of each stock and return them back to y.

    Here we used np.nanmean() function, in case some values would be nan.
    '''
    t0 = time.time()
    for i in range(40000-T): 
        
        for j in range(1082):
            y[i,j-1] = np.log(price[i+T,j-1]/price[i,j-1])
    t1 = time.time()
    y = y[:,0:1082]
    y_ = y.reshape(-1,1)
    
    # delete too-much nan value column
    # x = np.delete(x, 5)
    del address, x_i, x_, price, y
    gc.collect()
    print("y function running time should be, ", t1-t0)
    print("the shape of x is:", np.shape(x))
    print("the shape of y is:", y_.shape)
    return x, y_

In [42]:
x_, y_ = import_dataset()


(40000, 1082)
1 th factor have 0 number of nan factors 
(40000, 1082)
2 th factor have 0 number of nan factors 
(40000, 1082)
3 th factor have 0 number of nan factors 
(40000, 1082)
4 th factor have 0 number of nan factors 
(40000, 1082)
5 th factor have 0 number of nan factors 
(40000, 1082)
6 th factor have 0 number of nan factors 
(40000, 1082)
7 th factor have 0 number of nan factors 
(40000, 1082)
8 th factor have 0 number of nan factors 
(40000, 1082)
9 th factor have 0 number of nan factors 
(40000, 1082)
10 th factor have 0 number of nan factors 
(40000, 1082)
11 th factor have 0 number of nan factors 
(40000, 1082)
12 th factor have 0 number of nan factors 
(40000, 1082)
13 th factor have 0 number of nan factors 
(40000, 1082)
14 th factor have 0 number of nan factors 
(40000, 1082)
15 th factor have 0 number of nan factors 
(40000, 1082)
16 th factor have 0 number of nan factors 
(40000, 1082)
17 th factor have 0 number of nan factors 
(40000, 1082)
18 th factor have 2075839 

#### 2. Feature Engineering

Outline for possible solution:
1. Standardization
2. Develop Sharpe Ratio Criteria
3. Select best Ratios

##### 1. Standardization

In [65]:
def standardization(x):
    '''
    for every single factor, for every single time point, calculate prior 500 point's maximum and minimum, do standardization.
    '''
    
    # window size(for standardization)
    window = 500
    
    x = np.asarray(x)
    x = np.delete(x, [0, 1, 17], axis = 0)
    print('the shape of input matrix is %d, %d, %d' %(x.shape[0],x.shape[1], x.shape[2]))
    print('This matrix have %d nan value' %(np.isnan(x).sum()))
    
    
    t0 = time.time()
    x_bar = np.zeros((x.shape[0], (x.shape[1]-window), x.shape[2]))
    for i in range(x.shape[0]):
        print('%d th factor:'%i)
        for j in range(x.shape[1]-window):
            for k in range(x.shape[2]):
                x_ = x[i, j:j+500, k].reshape((500, 1))
                x_order = np.argsort(x_)
                max_x = x_[x_order[10]]
                min_x = x_[x_order[490]]
                assert max_x >= min_x
                if max_x - min_x != 0:
                    x_bar[i,j,k] = (x[i,j+500,k] - min_x)/(max_x - min_x)
                else:
                    x_bar[i,j,k] = 0
        gc.collect()
        '''

        for j in range(6):
            x_ = x[i * 1082: i* 1082 + 1082, j]
            x_order = np.argsort(x_)
            if i == 0:
                print(type(x_order))
                print(x_[x_order[108]]) # here we use 108
                print(x_[x_order[974]]) # here we use 1082 - 108 = 974 
            max_x = x_[x_order[54]] #there exist nan value in this np array
            min_x = x_[x_order[1028]]
            if max_x - min_x != 0:
                x_hat = (x_ - min_x)/(max_x - min_x) # this should be a range(0,1) standardization
                x_hat = 2 * x_hat - 1
                x_bar[i * 1082: i* 1082 + 1082, j] = x_hat
            else:
                x_hat = np.zeros(1082)
                x_bar[i * 1082: i* 1082 + 1082, j] = x_hat
        '''
        
    t1 = time.time()
    print('Total transfer time is %d' %(t1-t0))
    print(x_bar.shape)
    x_bar = np.asarray(x_bar)
    return x_bar


# But we should notice here that we don't need nan transfer. Instead, we should delete those lines with nan value
# No matter it's x or y.
def nan_transfer(x):
    where_are_nan = np.isnan(x)
    where_are_inf = np.isinf(x)
    x[where_are_nan] = 0
    x[where_are_nan] = 0
    return x

In [66]:
x__ = standardization(x_)
# x__ = nan_transfer(x__)

the shape of input matrix is 17, 40000, 1082
This matrix have 0 nan value
0 th factor:
1 th factor:
2 th factor:
3 th factor:
4 th factor:
5 th factor:
6 th factor:
7 th factor:
8 th factor:
9 th factor:
10 th factor:
11 th factor:
12 th factor:
13 th factor:
14 th factor:
15 th factor:
16 th factor:
Total transfer time is 6341
(17, 39500, 1082)


In [67]:
x_save = np.reshape(x__, [17,-1])

In [68]:
np.savetxt('../Data/x.csv', x_save)

In [69]:
np.savetxt('../Data/y.csv', y_)

#### 3. Splitting Data

In [72]:
def timing_splitter(x, y):
    '''
    T = 30
    training = 前19500
    testing = 后19970
    先截去后30个x
    截去前500个y
    再分
    train_x: 0到1082*19500
    train_y: 1082*500到1082*20000
    test_x: 1082*19500到1082*39470
    test_y: 1082*20000到1082*39970
    
    '''
    # define hyper-parameter:
    
    
    x = np.asarray(x)
    # x = np.delete(x, [3, 8, 16], axis = 0)
    y = np.asarray(y)
    x = x.reshape(17,-1).T
    y = y.reshape(-1,1)
    '''  
    # Cut first 30 rows of x
    # 30 * 1082 = 32460
    x = x[32460:,:]


    # transfer nan to 0
    x = np.nan_to_num(x)
    y = np.nan_to_num(y)
    '''
    print(np.shape(x))
    print(y.shape)
    
    # cutting first 
    train_X = x[:21099000,:]    # 3-D to 2-D
    train_y = y[541000:21640000,:]
    test_X = x[21099000:42706540,:]
    test_y = y[21640000:43247540,:]
    
    
    # deleting data if it have nan value inside of it.

    # deleting rows form train dataset
    train_index = []
    for i in range(train_y.shape[0]):
        if np.isnan(train_y[i]):
            train_index.append(i)
    train_X = np.delete(train_X, train_index, axis = 0)
    train_y = np.delete(train_y, train_index, axis = 0)
    
    # deleting rows form test dataset
    test_index = []
    for i in range(test_y.shape[0]):
        if np.isnan(test_y[i]):
            test_index.append(i)
    test_X = np.delete(test_X, test_index, axis = 0)
    test_y = np.delete(test_y, test_index, axis = 0)

    
    '''
    train_X= np.nan_to_num(train_X)
    train_y= np.nan_to_num(train_y)
    test_X= np.nan_to_num(test_X)
    test_y= np.nan_to_num(test_y)
    '''
    print("The shape of training set's X is,",train_X.shape)
    print("The shape of training set's y is,",train_y.shape)
    print("The shape of testing set's X is,",test_X.shape)
    print("The shape of testing set's y is,",test_y.shape)

    
    return train_X, train_y, test_X, test_y

In [73]:
train_X, train_y, test_X, test_y = timing_splitter(x__, y_)

(42739000, 17)
(43247540, 1)
The shape of training set's X is, (20106560, 17)
The shape of training set's y is, (20106560, 1)
The shape of testing set's X is, (21370430, 17)
The shape of testing set's y is, (21370430, 1)


In [74]:
gc.collect()

636

In [17]:
np.shape(x_roll)[0]

8

In [18]:
x_roll[1].shape

(5117240, 20)

#### 4. Model_fittig


Deep Learning Models Right Now!

使用的模型为：
1. Simple MLP
2. Ridge
3. <font color = #ff0000> KNN </font>
4. Lasso
5. Bayesian
6. <font color = #ff0000> MLP </font>
7. Decision Tree
8. Random Forest
9. <font color = #ff0000> SGD </font>
10. Gradient Boosting Decision Tree
11. AdaBoost
12. SVM regression

In [75]:
# Preparing for several judgement parameter



# Accuracy
from sklearn.metrics import accuracy_score as acc_s
from sklearn.metrics import mean_squared_error as mse
from scipy.stats import pearsonr
from sklearn.model_selection import GridSearchCV
from sklearn import cross_validation, metrics
'''
Since this is a regression problem instead of classification, 
we shouldn't use accuracy_score(compared one element from each other in this case .)
Use MSE and RMSE criteria instead.

'''

def rmse(y_test, y_pred):
    return np.sqrt(mse(y_test, y_pred))

def IC(x1, x2):
    pearson = []
    x1 = np.squeeze(x1)
    x2 = np.squeeze(x2)
    for i in range(int(np.shape(x1)[0]/1082)):
        x1_ = x1[i*1082:i*1082+1082,]
        x2_ = x2[i*1082:i*1082+1082,]
        pearson_ = pearsonr(x1_, x2_)[0]
        pearson.append(pearson_)
    pearson = np.asarray(pearson)
    pearson = np.squeeze(pearson)
    '''
    print(np.isnan(pearson).sum())
    print(pearson.shape)
    print(pearson.mean())
    print(pearson.std())
    '''
    temp = pearson.mean()/pearson.std()
    return temp

def nan_transfor(x):
    where_are_nan = np.isnan(x)
    where_are_inf = np.isinf(x)
    x[where_are_nan] = 0
    x[where_are_nan] = 0
    return x

def predictor(func):
    y_hat = func.predict(pred_x)
    y_hat = y_hat.reshape(40000,-1)
    return y_hat

def fitting(func, x_roll = x_roll, y_roll = y_roll, x__ = x__):
    '''
    diivding the dataset into 8 parts. Use former part as training dataset and latter part as testint dataset.
    '''
    t0 = time.time()
    print('model is %s' %func)
    # define several returnable parameters
    acc_1 = []    # mse
    acc_2 = []    # rmse
    acc_3 = []    # IC
    length = 40000 *1082 / 8    
    
    assert np.shape(x_roll)[0] == np.shape(y_roll)[0] == 8

    for i in range(np.shape(x_roll)[0]-1):
        
        train_X = np.asarray(x_roll[i])
        test_X = np.asarray(x_roll[i+1])
        train_y = np.asarray(y_roll[i])
        test_y = np.asarray(y_roll[i+1])
        print('%d th iteration'%i)
        print(test_X.shape[1])
        print(test_y.shape[0])
        assert train_X.shape[1] == 17
        assert train_X.shape[0] == train_y.shape[0]
        assert test_X.shape[0] == test_y.shape[0]
        
        '''
        我们的最终目标是返回一个40000*1082的matrix
        我们需要一组完整的x， 一组去掉nan的x， 一组去掉nan的y。
        去掉nan的x/y用来计算mse，rmse，IC index
        不去掉的用来计算整体，方法是如果是第一个model就计算10000*1082，其他的计算5000*1082
        '''
        func.fit(train_X, train_y)
        pred_y = func.predict(test_X)
        if pred_y.ndim == 1:
            pred_y = pred_y[:,np.newaxis]
        assert np.shape(pred_y)[0] == np.shape(test_y)[0]
        acc_1.append(mse(pred_y, test_y))
        acc_2.append(rmse(pred_y, test_y))
        acc_3.append(IC(pred_y, test_y))
        
        if i == 0:
            output = func.predict(x__[:int(2*length),:])
            if output.ndim == 1:
                output = output[:,np.newaxis]
            print('the %d th iteration. output shape is (%d,%d)' %(i, np.shape(output)[0], np.shape(output)[1]))
        if i != 0:
            output_ = func.predict(x__[int(length*(i+1)):int((i+2)*length),:])
            if output_.ndim == 1:
                output_ = output_[:,np.newaxis]
            print('the %d th iteration. output shape is (%d,%d)' %(i, np.shape(output_)[0], np.shape(output_)[1]))
            output = np.concatenate((output, output_))
    
    accuracy_1 = np.mean(acc_1)
    accuracy_2 = np.mean(acc_2)
    accuracy_3 = np.mean(acc_3)
    output = output.reshape(40000,-1)
    
    print('average value of mse is %f' %(accuracy_1))
    print('average value of rmse is %f' %(accuracy_2))
    print('average value of IC is %f' %(accuracy_3))
    
    t1 = time.time()
    print('Total time used for fitting model %s is: %f'%(func,(t1-t0)))
    
    return output, accuracy_1, accuracy_2, accuracy_3, (t1-t0)

In [76]:
'''
Clear content of all memories
'''
Method = []
MSE = []
RMSE = []
Time = []
y_hat = []
ic = []

##### a. Simple MLP

In [88]:
# Define a very simple 
model1 = Sequential()
model1.add(Dense(17, input_dim=17, kernel_initializer='normal', activation='sigmoid')) # relu will cause extra problem: loss = nan
model1.add(Dense(6, kernel_initializer='normal', activation='linear'))
model1.add(Dense(1, kernel_initializer='normal'))
model1.compile(loss='mse', optimizer=SGD(lr = 2e-4, momentum = 0.9, nesterov = True))

In [89]:
model1.fit(train_X, train_y, epochs = 15, batch_size = 128)

Epoch 1/15
20106560/20106560 [==============================] - 225s 11us/step - loss: 6.8838e-05
Epoch 2/15
20106560/20106560 [==============================] - 224s 11us/step - loss: 6.8817e-05
Epoch 3/15
20106560/20106560 [==============================] - 223s 11us/step - loss: 6.8817e-05
Epoch 4/15
20106560/20106560 [==============================] - 224s 11us/step - loss: 6.8817e-05
Epoch 5/15
20106560/20106560 [==============================] - 224s 11us/step - loss: 6.8817e-05
Epoch 6/15
20106560/20106560 [==============================] - 223s 11us/step - loss: 6.8817e-05
Epoch 7/15
20106560/20106560 [==============================] - 223s 11us/step - loss: 6.8817e-05
Epoch 8/15
20106560/20106560 [==============================] - 224s 11us/step - loss: 6.8817e-05
Epoch 9/15
20106560/20106560 [==============================] - 224s 11us/step - loss: 6.8817e-05
Epoch 10/15
20106560/20106560 [==============================] - 224s 11us/step - loss: 6.8817e-05
Epoch 11/15
2010656

In [92]:
# Define a very simple 
model2 = Sequential()
model2.add(Dense(17, input_dim=17, kernel_initializer='normal', activation='sigmoid')) # relu will cause extra problem: loss = nan
model2.add(Dense(64, kernel_initializer='normal', activation='linear'))
model2.add(Dense(32, kernel_initializer='normal', activation='linear'))
model2.add(Dense(1, kernel_initializer='normal'))
model2.compile(loss='mse', optimizer=SGD(lr = 2e-4, momentum = 0.9, nesterov = True))

In [94]:
model2.fit(train_X, train_y, epochs = 5, batch_size = 128)

Epoch 1/5
20106560/20106560 [==============================] - 242s 12us/step - loss: 6.8817e-05
Epoch 2/5
20106560/20106560 [==============================] - 243s 12us/step - loss: 6.8817e-05
Epoch 3/5
20106560/20106560 [==============================] - 248s 12us/step - loss: 6.8817e-05
Epoch 4/5
20106560/20106560 [==============================] - 249s 12us/step - loss: 6.8817e-05
Epoch 5/5
20106560/20106560 [==============================] - 262s 13us/step - loss: 6.8817e-05


In [96]:
score1 = model1.evaluate(test_X, test_y, batch_size = 128)

21370430/21370430 [==============================] - 107s 5us/step


In [100]:
score1

6.511823561204402e-05

In [98]:
score2 = model2.evaluate(test_X, test_y, batch_size = 128)
score2

21370430/21370430 [==============================] - 101s 5us/step


5.612874424704367e-05

In [107]:
from keras.layers.recurrent import GRU
from keras.layers.recurrent import SimpleRNN


In [108]:
# Define a very simple 
model3 = Sequential()
model3.add(SimpleRNN(32, activation = 'sigmoid', dropout = 0.1))
model3.add(Dense(128, input_dim=17, kernel_initializer='normal', activation='sigmoid')) # relu will cause extra problem: loss = nan
model3.add(Dense(64, kernel_initializer='normal', activation='linear'))
model3.add(Dense(1, kernel_initializer='normal'))
model3.compile(loss='mse', optimizer=SGD(lr = 2e-4, momentum = 0.9, nesterov = True))

In [109]:
model3.fit(train_X, train_y, epochs = 5, batch_size = 128)

ValueError: Input 0 is incompatible with layer simple_rnn_1: expected ndim=3, found ndim=2

##### b. Ridge

In [19]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha = 0.6)
y_hat_ridge, acc1, acc2, acc3, time_ = fitting(ridge)

Method.append('Ridge')
MSE.append(acc1)
RMSE.append(acc2)
ic.append(acc3)
Time.append(time_)


model is Ridge(alpha=0.6, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)
0 th iteration
17
5117240


/Users/kevin/anaconda3/lib/python3.6/site-packages/scipy/linalg/basic.py:40: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number/precision: 2.558700581455999e-25 / 1.1102230246251565e-16
  RuntimeWarning)


the 0 th iteration. output shape is (10820000,1)
1 th iteration
17
5185920


/Users/kevin/anaconda3/lib/python3.6/site-packages/scipy/linalg/basic.py:40: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number/precision: 1.88708247926861e-26 / 1.1102230246251565e-16
  RuntimeWarning)


the 1 th iteration. output shape is (5410000,1)
2 th iteration
17
5244720


...

the 4 th iteration. output shape is (5410000,1)
5 th iteration
17
5380160


/Users/kevin/anaconda3/lib/python3.6/site-packages/scipy/linalg/basic.py:40: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number/precision: 1.4155945094212022e-25 / 1.1102230246251565e-16
  RuntimeWarning)


the 5 th iteration. output shape is (5410000,1)
6 th iteration
17
5361870


/Users/kevin/anaconda3/lib/python3.6/site-packages/scipy/linalg/basic.py:40: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number/precision: 1.612231505477241e-25 / 1.1102230246251565e-16
  RuntimeWarning)


the 6 th iteration. output shape is (5410000,1)
average value of mse is 0.000059
average value of rmse is 0.007658
average value of IC is 1.270726
Total time used for fitting model Ridge(alpha=0.6, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001) is: 36.988617


##### c. Lasso

In [20]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha = 0.1)

y_hat_lasso, acc1, acc2, acc3, time_ = fitting(lasso)

Method.append('Lasso')
MSE.append(acc1)
RMSE.append(acc2)
ic.append(acc3)
Time.append(time_)


model is Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)
0 th iteration
17
5117240
the 0 th iteration. output shape is (10820000,1)
1 th iteration
17
5185920
the 1 th iteration. output shape is (5410000,1)
2 th iteration
17
5244720
the 2 th iteration. output shape is (5410000,1)
3 th iteration
17
5287280
the 3 th iteration. output shape is (5410000,1)
4 th iteration
17
5341120
the 4 th iteration. output shape is (5410000,1)
5 th iteration
17
5380160
the 5 th iteration. output shape is (5410000,1)
6 th iteration
17
5361870
the 6 th iteration. output shape is (5410000,1)
average value of mse is 0.000059
average value of rmse is 0.007668
average value of IC is 0.128804
Total time used for fitting model Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selecti

In [ ]:
Dict = {
    'mse':np.asarray(MSE),
    'rmse':np.asarray(RMSE),
    'time':np.asarray(Time),
    'IC':np.asarray(ic),
    # 'y_hat':y_hat
    # dataframe
}
result = pd.DataFrame(Dict, index = Method)


In [28]:
# Stacking zeros for type
zeros = np.zeros((40000, 747))
y_hat_linear = np.c_[y_hat_linear, zeros]
y_hat_ridge = np.c_[y_hat_ridge, zeros]
y_hat_lasso = np.c_[y_hat_lasso, zeros]


In [29]:
# Stacking zeros for type
zero = np.zeros((320, 1829))
y_hat_linear = np.r_[y_hat_linear, zero]
y_hat_ridge = np.r_[y_hat_ridge, zero]
y_hat_lasso = np.r_[y_hat_lasso, zero]


In [30]:
# Transfer float64 into float32
y_hat_linear = y_hat_linear.astype('float32')
y_hat_ridge = y_hat_ridge.astype('float32')
y_hat_lasso = y_hat_lasso.astype('float32')


In [36]:
# Output into requried type.
# 288 k kb
y_hat_linear.tofile('/opt/hhyang/6.29/rolling_linear/linear.bin')
y_hat_ridge.tofile('/opt/hhyang/6.29/rolling_linear/ridge.bin')
y_hat_lasso.tofile('/opt/hhyang/6.29/rolling_linear/lasso.bin')

In [37]:
def getStatsTS(X, Y, quantile=10, window=500, minCnt=250):
    """ 
    X: alpha
    Y: forward returns
    """
    #IC 
    X = np.asarray(X)
    Y = np.asarray(Y)
    if X.shape != Y.shape:
        raise
    N = len(X)
    IC = np.zeros((N,))

    bottom = 1.0 / quantile
    top = 1 - bottom

    # ts rank
    X = bn.move_rank(X, window=window, min_count=minCnt, axis=0)
    # norm to [0, 1]
    X = 0.5 * (X + 1)

    # get common data
    X = np.where((~np.isnan(X) & (~np.isnan(Y))), X, np.nan)
    Y = np.where((~np.isnan(X) & (~np.isnan(Y))), Y, np.nan)
    # cross-rank Y
    Y_rk = bn.nanrankdata(Y, axis=1)
    Y_rk /= bn.nanmax(Y_rk, axis=1)[:, np.newaxis]

    # ls
    LS = np.nanmean(np.where(X > top, Y, np.nan), axis=1) \
         - np.nanmean(np.where(X < bottom, Y, np.nan), axis=1)

    # Loop
    for ii in range(N):
        IC[ii] = np.corrcoef(X[ii][~np.isnan(X[ii])], Y_rk[ii][~np.isnan(Y_rk[ii])])[0,1]
            
        
    return IC, LS

In [35]:
Y_test = y_.reshape(-1, 1082)
print(Y_test.shape)
nans = np.nan*np.zeros((39970, 747))
nan = np.nan*np.zeros((350, 1829))
Y_test = np.c_[Y_test, nans]
Y_test = np.r_[Y_test, nan]
Y_test = Y_test.astype('float32')

(39970, 1082)


In [38]:
getStatsTS(y_hat_linear, Y_test)

/Users/kevin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in greater
/Users/kevin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: RuntimeWarning: Mean of empty slice
/Users/kevin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in less
/Users/kevin/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:1128: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/Users/kevin/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:73: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/Users/kevin/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3175: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/Users/kevin/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3109: RuntimeWarning: divide by zero encountered in double_sca

(array([nan, nan, nan, ..., nan, nan, nan]),
 array([nan, nan, nan, ..., nan, nan, nan], dtype=float32))

In [ ]:
k = 0

for i in range(10):
    for j in range(5):
        if k == 0:
        
        elif k != 0:
            
        k += 1